# Convolutional Neural Networks (CNNs)

### Welcome to the 10th Lab of 42028: Deep Learning and CNN!

In this  Lab/Tutorial session you will be implementing Inception module and create a Convolutional Neural Network for image classification .

So lets get started!

## Tutorial:
Implementation of Inception based Micro-GoogleNet CNN architecture using Keras for classfication of Cifar-10 dataset.

## Tasks for this week:

1. Implementation of Inception based Mini-GoogleNet CNN architecture for Cifar-10 classification using Keras API. 
2. Train and test model


**Reference and adapted from:** [https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/](https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/)

## The Inception Begins !

### Step 1: Import required packages

we will need tensorflow, numpy, os and keras


In [0]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf

from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

### Step-2: Design the Inception V1 module

<img src='http://drive.google.com/uc?export=view&id=12vu9O4-f-jeuQIWg6nwAUFiZV8qwKM9j' alt='Conv'>




In [0]:
# Create the Inception module
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    # 1X1 CONV
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    # 1X1 CONV --> 3x3 CONV
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    # 1X1 CONV --> 5x5 CONV
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    # 3X3 MAXPOOL --> 1X1 CONV
    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    # Concatenate the layers
    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

### Step 3: Design a Micro-GoogleNet with 1 Layer of Inception V1

<img src='http://drive.google.com/uc?export=view&id=143ci2dp1TKZb6o9fXXR3122ewmXcxreL' alt='Conv'>



In [0]:
# Create the input layer
input_layer = Input(shape=(32, 32, 3)) # Use the actual input size

# CONV Layer
x = Conv2D(64, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_1_3x3/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(1, 1), name='max_pool_1_3x3/2')(x)

#Adding Inception layer to te modeel
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(10, activation='softmax', name='output')(x)

In [0]:
# Form the model
model = Model(input_layer, x, name='inception_v1')

In [0]:
#Display Model Summary
model.summary()

### Step 4: Download the Cifar-10 dataset using keras

In [0]:
num_classes = 10 #Number of classes in the dataset

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()
    
    # Resize training images
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train, num_classes)
    Y_valid = np_utils.to_categorical(Y_valid, num_classes)
    
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    # preprocess data
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0

    return X_train, Y_train, X_valid, Y_valid

In [0]:
# Display the shapes of the training images
X_train, y_train, X_test, y_test = load_cifar10_data(32, 32)
print(X_train.shape)
print(X_test.shape)

In [0]:
## Display an image from the dataset
import matplotlib.pyplot as plt
plt.imshow(X_train[300])

### Step 5: Define the Hyper-parameters, Compile the model, Start training

In [0]:
#Define the number of epochs and learning rate
epochs = 25
initial_lrate = 0.01

#Create Learning rate decay
def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

#Define SGD paramenters
sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

## Compile Model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
#Start training the model
history1 = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=256, callbacks=[lr_sc])

In [0]:
## Plot the Traning and Validation loss
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

## Your task begins now!

## Task 1: Design a Mini-GoogleNet with Inception V1 model

### Add four inception layers in the Micro-GoogleNet architecture created above in the following order and configuration:

<img src='http://drive.google.com/uc?export=view&id=1674k5lS1gTnT7-YaqhoKEDQadttsJMcl' alt='Conv'>


In [0]:
## WRITE YOUR CODE HERE ## 
# Define the input layer
input_layer = 

# Add the first CONV layer and MaxPOOL layer
x = 
x = 

## Add Inception layer-1
x = 

## Add Inception layer-2
x = 

## Add MaxPool
x = 

## Add Inception layer-3
x = 

## Add Inception layer-4
x = 

## Add Avg Pool
x = 

## Add Dropout
x = 

## Add Dense layer
x = 

## END YOUR CODE HERE ## 

In [0]:
## WRITE YOUR CODE HERE ## 
# Form the Model
model = 

## END YOUR CODE HERE ##

In [0]:
# Print Model Summary
model.summary()

## Task 2: Define the HyperParamenters, Optimzer, etc

In [0]:
## WRITE YOUR CODE HERE ## 
#Define the number of epochs and learning rate
epochs =
initial_lrate = 
## END YOUR CODE HERE ##

#Create Learning rate decay
def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

## WRITE YOUR CODE HERE ##
#Define SGD paramenters
sgd = 

lr_sc = 
## END YOUR CODE HERE ##

## Task 3: Compile and Train the Model

In [0]:
# Compile the model and start training
## WRITE YOUR CODE HERE ##
model.compile() ## Add the missing code
history = model.fit() ## Add the missing code
## END YOUR CODE HERE ##

## Task 4: Test on Train and Test set

In [0]:
# Test the model on Test and Validation dataset
## WRITE YOUR CODE HERE ##
model.evaluate() ## Add the missing code for Training dataset
model.evaluate() ## Add the missing code for Test dataset
## END YOUR CODE HERE ##

## Task 5: Display the Train and Validation Loss curve

In [0]:
# Display the train and validation loss
## WRITE YOUR CODE HERE ##
loss = 
val_loss = 

epochs = 

## END YOUR CODE HERE ##

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
